In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

from sklearn.linear_model import LinearRegression #For regression model
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [3]:
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [4]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [5]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [6]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [7]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [8]:
print('training set prep')
train_df=seperate_monthly(train_df)
print('test set prep')
test_df=seperate_monthly(test_df)

training set prep
test set prep


In [9]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [10]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [11]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [12]:
Regresion_model= model = LinearRegression() #Define model object

In [13]:
X_train_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
77618,0.206578,0.363636,0.366667,0.148286,0.087306,0.206495,0.092543,0.192364,0.102937,0.098827,...,-0.401198,0.215889,-0.109288,-0.091063,-0.396574,0.865059,0.023280,-0.243063,0.083333,0.9375
160611,0.427460,0.545455,0.400000,0.026309,-0.159091,-0.087150,-0.164420,-0.174585,-0.119720,-0.105867,...,0.043267,0.070946,-0.395088,-0.233314,-0.511917,-0.050410,0.896345,0.188134,0.166667,1.0000
238880,0.635771,0.727273,0.100000,0.097119,0.089523,0.104080,0.101527,0.044885,0.102937,0.051369,...,-0.514169,0.871513,-0.032319,-0.379996,0.075442,0.258979,0.636494,0.365892,0.250000,1.0000
145302,0.386716,0.500000,0.666667,0.130660,-0.040743,0.006661,-0.016173,-0.059166,0.066853,0.106128,...,-0.246291,0.436247,-0.186854,-0.376320,0.075867,-0.015649,0.574327,-0.075570,0.250000,1.0000
318065,0.846519,0.909091,0.033333,0.037067,0.276053,0.300583,0.278826,0.327018,0.251189,0.312647,...,0.305059,0.151155,0.070505,0.752545,-0.662588,0.183556,-0.466120,0.504286,0.916667,0.8750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221739,0.590150,0.681818,0.266667,0.210718,0.102827,0.129892,0.087451,0.186535,0.211748,0.090482,...,-0.145493,0.017001,-0.219869,-0.278281,0.206640,-0.015136,-0.014046,0.421272,0.416667,0.9375
98512,0.262186,0.409091,0.400000,0.387119,0.042960,0.112684,0.074274,0.039930,0.074965,0.057106,...,0.038513,0.389058,0.307677,-0.231712,0.113748,0.042201,0.372220,0.115836,0.250000,1.0000
23417,0.062324,0.181818,1.000000,0.209376,0.781319,0.799889,0.768793,0.773535,0.677203,0.750978,...,-0.010460,0.024956,-0.835720,-0.257353,-0.166099,0.270010,-0.238523,-0.228965,0.750000,0.8750
341379,0.908568,0.954545,0.166667,0.345571,0.459812,0.551762,0.458521,0.587875,0.480559,0.543155,...,0.066470,0.350480,0.049821,0.457485,0.204405,-0.425346,0.196515,0.054365,0.750000,0.8750


In [14]:
model.fit(X_train_df, y_train_df)

LinearRegression()

In [15]:
predictions=model.predict(X_train_df)

In [16]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [17]:
i=375734
final['index']=0
while i<407088:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,5.972525,375734
1,1.597201,375735
2,7.293726,375736
3,5.855379,375737
4,4.798767,375738
...,...,...
352599,11.913198,0
352600,9.081151,0
352601,23.471560,0
352602,14.933821,0


In [18]:
final.to_csv('/kaggle/working/regression_wetherprediction.csv',index=False)